# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [4]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target, columns=['price'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
linreg.fit(X_train, y_train)
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)
train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test
mse_train = np.sum((y_train-y_hat_train)**2)/len(y_train)
mse_test =np.sum((y_test-y_hat_test)**2)/len(y_test)

Fit the model and apply the model to the make test set predictions

Calculate the residuals and the mean squared error

In [12]:
print('Train Mean Squarred Error:', mse_train)
print('Test Mean Squarred Error:', mse_test)

Train Mean Squarred Error: price    19.920831
dtype: float64
Test Mean Squarred Error: price    29.867131
dtype: float64


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [46]:
blah = [pd.DataFrame(columns=df_concat.columns) for _ in range(10)]

for i in range(len(df_concat)):
    print(df_concat.iloc[i])
    if i>10: break

blah

CHAS      0.000000
RM        6.575000
DIS       0.542096
B         1.000000
LSTAT    -1.275260
price    24.000000
Name: 0, dtype: float64
CHAS      0.000000
RM        6.421000
DIS       0.623954
B         1.000000
LSTAT    -0.263711
price    21.600000
Name: 1, dtype: float64
CHAS      0.000000
RM        7.185000
DIS       0.623954
B         0.989737
LSTAT    -1.627858
price    34.700000
Name: 2, dtype: float64
CHAS      0.000000
RM        6.998000
DIS       0.707895
B         0.994276
LSTAT    -2.153192
price    33.400000
Name: 3, dtype: float64
CHAS      0.000000
RM        7.147000
DIS       0.707895
B         1.000000
LSTAT    -1.162114
price    36.200000
Name: 4, dtype: float64
CHAS      0.000000
RM        6.430000
DIS       0.707895
B         0.992990
LSTAT    -1.200048
price    28.700000
Name: 5, dtype: float64
CHAS      0.000000
RM        6.012000
DIS       0.671500
B         0.996722
LSTAT     0.248456
price    22.900000
Name: 6, dtype: float64
CHAS      0.000000
RM        6.172

[Empty DataFrame
 Columns: [CHAS, RM, DIS, B, LSTAT, price]
 Index: [], Empty DataFrame
 Columns: [CHAS, RM, DIS, B, LSTAT, price]
 Index: [], Empty DataFrame
 Columns: [CHAS, RM, DIS, B, LSTAT, price]
 Index: [], Empty DataFrame
 Columns: [CHAS, RM, DIS, B, LSTAT, price]
 Index: [], Empty DataFrame
 Columns: [CHAS, RM, DIS, B, LSTAT, price]
 Index: [], Empty DataFrame
 Columns: [CHAS, RM, DIS, B, LSTAT, price]
 Index: [], Empty DataFrame
 Columns: [CHAS, RM, DIS, B, LSTAT, price]
 Index: [], Empty DataFrame
 Columns: [CHAS, RM, DIS, B, LSTAT, price]
 Index: [], Empty DataFrame
 Columns: [CHAS, RM, DIS, B, LSTAT, price]
 Index: [], Empty DataFrame
 Columns: [CHAS, RM, DIS, B, LSTAT, price]
 Index: []]

In [83]:
def kfolds(data, k):
    # Force data as pandas dataframe
    # add 1 to fold size to account for leftovers
    data = pd.DataFrame(data)
    n = len(data)
    fold_n = n // k
    remainder = n%k
    folds = [pd.DataFrame(columns=data.columns) for _ in range(k)]
    foldstart = 0
    for i in range(k):
        if i<remainder:
            folds[i] = data.iloc[foldstart:(foldstart+fold_n+1), :]
        else:
            folds[i] = data.iloc[foldstart:(foldstart+fold_n), :]
        foldstart += fold_n
    return folds

### Apply it to the Boston Housing Data

In [84]:
# Make sure to concatenate the data again
df_concat = pd.concat([X, y], axis=1)
df_concat.head()

,CHAS,RM,DIS,B,LSTAT,price
0,0.0,6.575,0.542096,1.000000,-1.275260,24.0
1,0.0,6.421,0.623954,1.000000,-0.263711,21.6
2,0.0,7.185,0.623954,0.989737,-1.627858,34.7
3,0.0,6.998,0.707895,0.994276,-2.153192,33.4
4,0.0,7.147,0.707895,1.000000,-1.162114,36.2


In [85]:
len(df_concat)

506

In [89]:
boston_folds = kfolds(df_concat, 5)
len(boston_folds)
for fold in boston_folds:
    print(len(fold))

102
101
101
101
101


### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [97]:
test_errs = []
train_errs = []
k=5



for n in range(k):
    # Split in train and test for the fold
    adj_folds = boston_folds.copy()
    test = adj_folds.pop(n)
    train = pd.concat(adj_folds)
    # Fit a linear regression model
    X_train = train[X.columns]
    y_train = train[y.columns]
    X_test = test[X.columns]
    y_test = test[y.columns]
    linreg.fit(X_train, y_train)
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    train_residuals = y_hat_train - y_train
    test_residuals = y_hat_test - y_test
    mse_train = np.sum((y_train-y_hat_train)**2)/len(y_train)
    mse_test =np.sum((y_test-y_hat_test)**2)/len(y_test)
    train_errs.append(mse_train)
    test_errs.append(mse_test)
    #Evaluate Train and Test Errors


print(train_errs)
print(test_errs)

[price    23.663952
dtype: float64, price    22.622037
dtype: float64, price    19.184949
dtype: float64, price    14.966882
dtype: float64, price    22.303315
dtype: float64]
[price    13.675297
dtype: float64, price    17.036983
dtype: float64, price    37.612544
dtype: float64, price    57.454979
dtype: float64, price    24.743414
dtype: float64]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [100]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error")


Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [101]:
print(cv_5_results)

[-13.40514492 -17.4440168  -37.03271139 -58.27954385 -26.09798876]


##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!